In [21]:
import argparse
import datetime
import os
import sys
import timeit
import warnings

import SimpleITK as sitk
import copy
import itertools
import SimpleITK as sitk
import numpy as np

if sys.version_info[0:2] > (3, 9):
    raise Exception(
        "Python version above 3.9 may cause problems with SimpleITK. [BufferError: memoryview has 1 exported buffer]"
    )

try:
    import mialab.data.structure as structure
except ImportError:
    # Append the MIALab root directory to Python path
    sys.path.insert(0, os.path.join(os.path.dirname(sys.argv[0]), ".."))
    import mialab.data.structure as structure


In [136]:
def subtract_overlap(larger_structure, smaller_structure):
    # Convert images to arrays
    larger_array = sitk.GetArrayFromImage(larger_structure)
    smaller_array = sitk.GetArrayFromImage(smaller_structure)

    # Identify overlapping regions
    overlap_array = np.logical_and(larger_array > 0, smaller_array > 0)

    # Set overlapping parts to 0 in the larger structure
    larger_array = np.where(overlap_array, 0, larger_array)

    # Identify non-overlapping regions and keep them in the larger structure
    larger_array = np.where(np.logical_not(overlap_array), larger_array, 0)

    return sitk.GetImageFromArray(larger_array)


In [141]:
# Replace these paths with the actual paths to your segmentation files
white_matter_path = "fromubelix/white_matter/117122_SEG.mha"
gray_matter_path = "fromubelix/grey_matter/117122_SEG.mha"
thalamus_path = "fromubelix/thalamus_matter/117122_SEG.mha"
hippocampus_path = "fromubelix/hippocampus_matter/117122_SEG.mha"
amygdala_path = "fromubelix/amygdala_matter/117122_SEG.mha"

# Read segmentation images
white_matter = sitk.ReadImage(white_matter_path)
gray_matter = sitk.ReadImage(gray_matter_path)
thalamus = sitk.ReadImage(thalamus_path)
hippocampus = sitk.ReadImage(hippocampus_path)
amygdala = sitk.ReadImage(amygdala_path)

# Copy original images
white_matter_original = copy.deepcopy(white_matter)
gray_matter_original = copy.deepcopy(gray_matter)
thalamus_original = copy.deepcopy(thalamus)
hippocampus_original = copy.deepcopy(hippocampus)
amygdala_original = copy.deepcopy(amygdala)

# priority list 
priority_list = ['hypocampus', 'amygdala', 'thalamus', 'gray_matter', 'white_matter']

# substract overlap from larger structures
# white matter is substracted gray matter, thalamus, hippocampus, amygdala
# gray matter is substracted thalamus, hippocampus, amygdala
# thalamus is substracted hippocampus, amygdala
# amygdala is substracted hippocampus

# white matter
white_matter = subtract_overlap(white_matter, gray_matter)
white_matter = subtract_overlap(white_matter, thalamus)
white_matter = subtract_overlap(white_matter, hippocampus)
white_matter = subtract_overlap(white_matter, amygdala)

# gray matter
gray_matter = subtract_overlap(gray_matter, thalamus)
gray_matter = subtract_overlap(gray_matter, hippocampus)
gray_matter = subtract_overlap(gray_matter, amygdala)

# thalamus
thalamus = subtract_overlap(thalamus, hippocampus)
thalamus = subtract_overlap(thalamus, amygdala)

# amygdala
amygdala = subtract_overlap(amygdala, hippocampus)

# Copy information from original images
white_matter.CopyInformation(white_matter_original)
gray_matter.CopyInformation(gray_matter_original)
thalamus.CopyInformation(thalamus_original)
hippocampus.CopyInformation(hippocampus_original)
amygdala.CopyInformation(amygdala_original)

# Save the images
sitk.WriteImage(white_matter, "overlapping_output/117122/white_matter_r.mha")
sitk.WriteImage(gray_matter, "overlapping_output/117122/gray_matter_r.mha")
sitk.WriteImage(thalamus, "overlapping_output/117122/thalamus_r.mha")
sitk.WriteImage(hippocampus, "overlapping_output/117122/hippocampus_r.mha")
sitk.WriteImage(amygdala, "overlapping_output/117122/amygdala_r.mha")